# Python tutorial: kaip išsaugoti funkcijas ir jas naudoti kituose skaičiavimuose

## Tikslas

- Parodyti, kaip funkcijos veikia notebook aplinkoje ir kada jos yra „pasiekiamos“.
- Pademonstruoti, kaip funkcijas perkelti į atskirą `.py` failą (modulį).
- Parodyti, kaip importuoti funkcijas iš modulio į notebook ir naudoti pakartotinai.
- Paminėti dažniausias klaidas: nepaleistas langelis su `def`, neteisingas importas, failo vieta, talpyklos (cache) efektas.

Pastaba: pavyzdžiuose naudojamos paprastos analitinės funkcijos (pardavimai, PVM).

## 1) Funkcijų „gyvavimo“ taisyklė notebook'e

Funkcija, aprašyta su `def`, tampa prieinama tik tada, kai langelis su jos aprašymu yra paleistas.

- Jei notebook perkraunamas (Restart Kernel), visos anksčiau aprašytos funkcijos iš atminties dingsta.
- Po perkrovimo reikia vėl paleisti langelius su `def`.

In [ ]:
# Funkcijos aprašymas (pirmas kartas)
def add_vat(amount, vat_rate=0.21):
    return round(amount * (1 + vat_rate), 2)

# Funkcijos panaudojimas (tame pačiame notebook'e, žemiau)
print(add_vat(1000))
print(add_vat(1000, vat_rate=0.09))

### Dažna klaida: funkcija nenaudojama, nes langelis su `def` nebuvo paleistas

Tokiu atveju gaunamas `NameError`.
Žemiau pateikiamas pavyzdys kaip atrodo klaida (palikti kaip komentarą).

In [ ]:
# Jei add_vat funkcijos langelis nebuvo paleistas, įvyktų klaida:
# print(add_vat(100))  # NameError: name 'add_vat' is not defined

print("Pastaba: NameError pasireiškia, kai funkcija dar neapibrėžta atmintyje.")

## 2) Funkcijų pernaudojimas: naudoti funkcijas kituose skaičiavimuose

Kartą aprašytą funkciją galima naudoti:
- cikluose,
- su sąrašais ir žodynais,
- kitose funkcijose.

In [ ]:
def line_total(qty, unit_price, discount=0.0):
    return round(qty * unit_price * (1 - discount), 2)

sales_lines = [
    {"product": "Milk", "qty": 2, "unit_price": 1.50, "discount": 0.00},
    {"product": "Bread", "qty": 1, "unit_price": 0.99, "discount": 0.10},
    {"product": "Eggs", "qty": 3, "unit_price": 0.80, "discount": 0.00},
    {"product": "Chocolate", "qty": 4, "unit_price": 2.99, "discount": 0.07}
]

net_total = 0.0
for r in sales_lines:
    net_total += line_total(r["qty"], r["unit_price"], r["discount"])

gross_total = add_vat(net_total)

print("Suma be PVM:", net_total)
print("Suma su PVM:", gross_total)

## 3) Kodėl verta išsaugoti funkcijas atskirame faile

Kai analizės užduočių daugėja, funkcijas patogu laikyti vienoje vietoje, pavyzdžiui, faile `calculations.py`.

Privalumai:
- nereikia kopijuoti funkcijų į kiekvieną notebook'ą,
- taisyklės laikomos vienoje vietoje,
- pasikeitus logikai (pvz., PVM tarifui), pakeitimas atliekamas vieną kartą.

## 4) Sukurti modulį `.py` faile iš notebook'o

Žemiau sukuriamas failas `calculations.py` tame pačiame aplanke, kuriame yra notebook'as.

Pastaba:
- Jei notebook'as saugomas kitoje vietoje, modulis turi būti sukurtas tame pačiame kataloge arba įtrauktas į Python paieškos kelią.

In [ ]:
module_code = '''
def line_total(qty, unit_price, discount=0.0):
    return round(qty * unit_price * (1 - discount), 2)

def add_vat(amount, vat_rate=0.21):
    return round(amount * (1 + vat_rate), 2)

def revenue_by_channel(lines):
    result = {}
    for row in lines:
        ch = row.get("channel", "unknown")
        if not isinstance(ch, str) or ch.strip() == "":
            ch = "unknown"
        ch = ch.strip().lower()

        amount = line_total(row["qty"], row["unit_price"], row.get("discount", 0.0))
        result[ch] = result.get(ch, 0.0) + amount

    for k in result:
        result[k] = round(result[k], 2)
    return result
'''.strip()

with open("calculations.py", "w", encoding="utf-8") as f:
    f.write(module_code)

print("Sukurtas failas: calculations.py")

## 5) Importuoti funkcijas iš modulio

Po to, kai modulis sukurtas, galima importuoti konkrečias funkcijas:

- `from calculations import add_vat`
- `from calculations import line_total, revenue_by_channel`

In [2]:
from calculations import add_vat, line_total, revenue_by_channel

print(add_vat(10000))
print(line_total(21, 71.50))

12100.0
1501.5


## 6) Naudoti importuotas funkcijas su verslo duomenimis

Pavyzdys:
- turėti pardavimų eilutes su kanalais,
- suskaičiuoti pajamas pagal kanalą,
- rasti didžiausią kanalą su built-in `max()`.

In [3]:
sales_lines2 = [
    {"channel": "online", "product": "Milk", "qty": 2, "unit_price": 1.50, "discount": 0.00},
    {"channel": "online", "product": "Bread", "qty": 1, "unit_price": 0.99, "discount": 0.10},
    {"channel": "direct", "product": "Eggs", "qty": 3, "unit_price": 0.80, "discount": 0.00},
    {"channel": "partner", "product": "Butter", "qty": 1, "unit_price": 2.20, "discount": 0.05},
]

rev_ch = revenue_by_channel(sales_lines2)
print(rev_ch)

best = max(rev_ch.items(), key=lambda kv: kv[1])
print("Didžiausias kanalas:", best)

{'online': 3.89, 'direct': 2.4, 'partner': 2.09}
Didžiausias kanalas: ('online', 3.89)


## 7) Dažna klaida: pakeistas modulis, bet importas „neatsinaujina“

Notebook aplinkoje (Jupyter) importas dažnai būna „užtalpinamas“ (cached).

Jei `calculations.py` pakeičiamas, o po to tiesiog vėl parašomas `from calculations import ...`,
gali būti, kad bus naudojama sena versija.

Tokiu atveju naudojamas `importlib.reload()`.

In [4]:
import calculations
import importlib

importlib.reload(calculations)

print("Modulis perkrautas:", calculations.add_vat(100))

Modulis perkrautas: 121.0


## 8) Dažna klaida: modulis nerandamas (ModuleNotFoundError)

Priežastys:
- `calculations.py` yra kitame aplanke nei notebook'as.
- Darbo katalogas (working directory) nėra tas pats aplankas.

Sprendimas:
- patikrinti dabartinį darbo katalogą,
- įsitikinti, kad `calculations.py` yra tame pačiame aplanke,
- prireikus pakeisti darbo katalogą arba pakoreguoti `sys.path`.

In [5]:
import os
print("Darbo katalogas:", os.getcwd())
print("Aplanko failai:", os.listdir("."))

Darbo katalogas: c:\Users\User\OneDrive\Desktop\BIT\2025 11 12 DUOM\V Modulis_Python_2025 11 12 DUOM\BIT_PYTHON\Paskaita_5
Aplanko failai: ['calculations.py', 'data.txt', 'excel file.ipynb', 'NumPy', 'numpy_advanced_analizei.ipynb', 'NumPy_Data Analytics.ipynb', 'numpy_duomenu_analizei.ipynb', 'numpy_funkcijos_duomenu_analizei.ipynb', 'Pamoka_5.ipynb', 'python_args_kwargs_business_hr_lt.ipynb', 'python_args_kwargs_tutorial_lt.ipynb', 'python_duomenu_tipai_pardavimuose.ipynb', 'python_functions_business_use_cases_lt.ipynb', 'python_functions_def_tutorial_lt.ipynb', 'python_lambda_functions_tutorial_lt.ipynb', 'python_lazy_iterators_detailed_with_def_lambda_lt.ipynb', 'python_save_and_reuse_functions_lt.ipynb', 'ski_shop_data.xlsx', '__pycache__']


## 9) Praktinis patarimas struktūrai

Patogu turėti struktūrą:

- `notebooks/` (analizės ir demonstracijos)
- `modules/` arba `src/` (funkcijos ir pagalbiniai įrankiai)
- `data/` (duomenų failai)

Tuomet importams dažniausiai naudojamas aiškus kelias ir moduliai laikomi atskirai nuo analizės.

## Santrauka

- Notebook'e funkcija tampa pasiekiama tik paleidus langelį su `def`.
- Funkcijas patogu laikyti `.py` faile ir importuoti į skirtingus notebook'us.
- Pakeitus modulį, kartais reikia `importlib.reload()` dėl importo talpyklos.
- `ModuleNotFoundError` dažniausiai reiškia netinkamą failo vietą arba darbo katalogą.